In [18]:
from torchvision.datasets import ImageFolder
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm

from datetime import datetime
import pytz 

num_classes = 16
frame_size = (512, 512)
batch_size = 32
weights_save_dir = "/media/viplab/DATADRIVE1/driver_action_recognition/resnext_weights/dash"
view = "dash"
all_images_paths = "/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_dash"
log_file_path = "/media/viplab/DATADRIVE1/driver_action_recognition/resnext_weights/dash/dash_logs_resnet.txt"

ist = pytz.timezone('Asia/Kolkata')

# image_paths = []
# labels = []
image_paths_train = []
labels_train = []
image_paths_test = []
labels_test = []
user_ids = set()


transform = transforms.Compose([
        transforms.Resize(frame_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
])


class VideoDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for class_idx, class_dir in enumerate(os.listdir(root_dir)):
            class_path = os.path.join(root_dir, class_dir)
            if os.path.isdir(class_path):
                for user_dir in os.listdir(class_path):
                    user_path = os.path.join(class_path, user_dir)
                    if os.path.isdir(user_path):
                        for img_file in os.listdir(user_path):
                            img_path = os.path.join(user_path, img_file)
                            if img_file.endswith(('.png', '.jpg', '.jpeg')):
                                self.image_paths.append(img_path)
                                self.labels.append(class_idx)  # Class label based on folder name

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('L')  # Convert to greyscale (L mode)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


class GreyscaleImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # '/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_dash/class_15/user_id_25432/Dashboard_user_id_25432_5_6919.jpg
        dimg_path = self.image_paths[idx]
        label = self.labels[idx]
        parts = dimg_path.split("/")
        rimg_path = "/".join(parts[:6]) + "/cut_frames_rear/" + "/".join(parts[7:-1]) + "/" + parts[-1].replace("Dashboard_user_id", "Rear_view_user_id")
        simg_path = "/".join(parts[:6]) + "/cut_frames_side/" + "/".join(parts[7:-1]) + "/" + parts[-1].replace("Dashboard_user_id", "Right_side_window_user_id")
        image_d = Image.open(dimg_path).convert('L')  # Convert to greyscale
        image_r = Image.open(rimg_path).convert('L')
        image_s = Image.open(simg_path).convert('L')
        if self.transform:
            image_d = self.transform(image_d)
            image_r = self.transform(image_r)
            image_s = self.transform(image_s)

        return image_d, image_r, image_s, label


for class_folder in os.listdir(all_images_paths):
    class_folder_path = os.path.join(all_images_paths, class_folder)
    if os.path.isdir(class_folder_path):
        for user_folder in os.listdir(class_folder_path):
            user_folder_path = os.path.join(class_folder_path, user_folder)
            if os.path.isdir(user_folder_path):
                user_ids.add(user_folder)


user_ids = sorted(user_ids)
#First 60
train_users = user_ids[:60]
#Last 9
test_users = user_ids[60:]   # Last 9 users for testing

for class_folder in os.listdir(all_images_paths):
    class_folder_path = os.path.join(all_images_paths, class_folder)
    if os.path.isdir(class_folder_path):
        label = int(class_folder.split('_')[1])  # Extract class number
        for user_folder in os.listdir(class_folder_path):
            user_folder_path = os.path.join(class_folder_path, user_folder)
            if os.path.isdir(user_folder_path):
                for img_file in os.listdir(user_folder_path):
                    if img_file.endswith(('png', 'jpg', 'jpeg')):
                        img_path = os.path.join(user_folder_path, img_file)
                        if user_folder in train_users:
                            image_paths_train.append(img_path)
                            labels_train.append(label)
                        elif user_folder in test_users:
                            image_paths_test.append(img_path)
                            labels_test.append(label)

X_train = image_paths_train
y_train = labels_train
X_test = image_paths_test
y_test = labels_test



train_dataset = GreyscaleImageDataset(X_train, y_train, transform=transform)
val_dataset = GreyscaleImageDataset(X_test, y_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


      

In [3]:
X_train

['/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_dash/class_15/user_id_25432/Dashboard_user_id_25432_5_6919.jpg',
 '/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_dash/class_15/user_id_25432/Dashboard_user_id_25432_7_7021.jpg',
 '/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_dash/class_15/user_id_25432/Dashboard_user_id_25432_5_7309.jpg',
 '/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_dash/class_15/user_id_25432/Dashboard_user_id_25432_5_6901.jpg',
 '/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_dash/class_15/user_id_25432/Dashboard_user_id_25432_7_7003.jpg',
 '/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_dash/class_15/user_id_25432/Dashboard_user_id_25432_5_6781.jpg',
 '/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_dash/class_15/user_id_25432/Dashboard_user_id_25432_7_7309.jpg',
 '/media/viplab/Storage1/driver_action_recogniti

In [15]:
dimg_path = X_train[0]
parts = dimg_path.split("/")
rimg_path = "/".join(parts[:6]) + "/cut_frames_rear/" + "/".join(parts[7:-1]) + "/" + parts[-1].replace("Dashboard_user_id", "Rear_view_user_id")
simg_path = "/".join(parts[:6]) + "/cut_frames_side/" + "/".join(parts[7:-1]) + "/" + parts[-1].replace("Dashboard_user_id", "Right_side_window_user_id")

In [17]:
simg_path

'/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_side/class_15/user_id_25432/Right_side_window_user_id_25432_5_6919.jpg'

In [6]:
dimg_path

'/media/viplab/Storage1/driver_action_recognition/crop_new/cut_frames_dash/class_15/user_id_25432/Dashboard_user_id_25432_5_6919.jpg'

In [ ]:
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        self.modeld = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V2)
        self.modelr = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V2)
        self.models = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V2)

        first_conv_layer = self.model.conv1
        new_first_layer = nn.Conv2d(
            in_channels=1,  # Change from 3 to 1 for grayscale input
            out_channels=first_conv_layer.out_channels,
            kernel_size=first_conv_layer.kernel_size,
            stride=first_conv_layer.stride,
            padding=first_conv_layer.padding,
            bias=False
        )

        with torch.no_grad():
            new_first_layer.weight[:] = first_conv_layer.weight.mean(dim=1, keepdim=True)

        self.modeld.conv1 = new_first_layer 
        self.modelr.conv1 = new_first_layer 
        self.models.conv1 = new_first_layer 

        num_features = self.model.fc.in_features * 3

        self.modeld.fc = nn.Identity()
        self.modelr.fc = nn.Identity()
        self.models.fc = nn.Identity()

        self.classifier = nn.Linear(num_features, num_classes)

    def forward(self, x1, x2, x3):
        x1 = self.modeld(x1)
        x2 = self.modelr(x2)
        x3 = self.models(x3)
        x = torch.cat([x1, x2, x3])
        x = self.classifier(x)
        return x
 